In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tqdm.notebook import tqdm

from pymorphy2 import MorphAnalyzer

# BAZA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from argparse import Namespace
from collections import Counter
import string
import os
# OSNOVA
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from torchmetrics.functional import r2_score
from sklearn.model_selection import train_test_split
import fasttext
import fasttext.util
import random
from catboost import CatBoostRegressor
from catboost.metrics import R2
from catboost import Pool

seed = 1000

In [39]:
train_df = pd.read_csv('data/vacancies_train.csv')
test_df = pd.read_csv('data/vacancies_test.csv')

In [40]:
cleaned_train = pd.read_csv('data/preprocessed_data_train.csv')
cleaned_test = pd.read_csv('data/preprocessed_data_test.csv')

In [41]:
additional_train = pd.read_csv("data/additional_features_train.csv")
additional_test = pd.read_csv("data/additional_features_test.csv")

In [42]:
fasttext_transformed_sentences_train = pd.read_csv("data/fasttext_train.csv")
fasttext_transformed_sentences_test = pd.read_csv("data/fasttext_test.csv")

In [51]:
def seed_everywhere(seed=1000):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [52]:
seed_everywhere(seed)

In [53]:
additional_train = additional_train[train_df.duplicated() == False]
cleaned_train = cleaned_train[train_df.duplicated() == False]
fasttext_transformed_sentences_train = fasttext_transformed_sentences_train[train_df.duplicated() == False]
train_df = train_df[train_df.duplicated() == False]

additional_train = additional_train.reset_index(drop=True)
cleaned_train = cleaned_train.reset_index(drop=True)
fasttext_transformed_sentences_train = fasttext_transformed_sentences_train.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

train_df.duplicated().sum()

0

In [54]:
additional_train = additional_train.drop(train_df.query("salary < 100").index)
cleaned_train = cleaned_train.drop(train_df.query("salary < 100").index)
fasttext_transformed_sentences_train = fasttext_transformed_sentences_train.drop(train_df.query("salary < 100").index)
train_df = train_df.drop(train_df.query("salary < 100").index)

In [55]:
additional_train = additional_train.reset_index(drop=True)
cleaned_train = cleaned_train.reset_index(drop=True)
fasttext_transformed_sentences_train = fasttext_transformed_sentences_train.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

In [56]:
cat_features = ["experience", "schedule", "employment"]

In [59]:
train_df['cleaned'] = cleaned_train.text
test_df['cleaned'] = cleaned_test.text

In [60]:
train_df

,name,experience,schedule,employment,description,key_skills,salary,cleaned
0,Водитель-экспедитор,От 3 до 6 лет,Полный день,Полная занятость,"Водитель-экспедитор категории В,С (а\м ГАЗЕЛЬ ...",[],42500.0,водитель экспедитор категория м газель тент га...
1,Кладовщик-комплектовщик/оператор 1С,От 1 года до 3 лет,Полный день,Полная занятость,Обязанности: Списывание материалов в произво...,"['знание пк', 'работа в 1С', 'Работа в команде...",50000.0,обязанность списывание материал производство п...
2,"Продавец магазина ""Магнит"" (Кировский р-он, В....",Нет опыта,Сменный график,Полная занятость,"В крупнейшую Федеральную сеть магазинов ""Магни...",[],31500.0,крупный федеральный сеть магазин магнит требов...
3,Администратор магазина (ТЦ Spar),От 1 года до 3 лет,Полный день,Полная занятость,Крупная федеральная сеть магазинов мужской и ж...,[],36700.0,крупный федеральный сеть магазин мужской женск...
4,Мойщик посуды (кулинария),Нет опыта,Сменный график,Полная занятость,Обязанности: Уборка производственных площадей...,"['Пользователь ПК', 'Грамотная речь', 'Работа ...",37000.0,обязанность уборка производственный площадь ку...
...,...,...,...,...,...,...,...,...
16311,Полицейский специального полка полиции,Нет опыта,Полный день,Полная занятость,Обязанности: обеспечение общественного поряд...,"['Грамотная речь', 'Обучение и развитие', 'Упр...",67500.0,обязанность обеспечение общественный порядок о...
16312,Врач стоматолог-ортопед (г. Новосибирск),Нет опыта,Полный день,Полная занятость,Обязанности: Оказывает квалифицированную меди...,['Медицинская документация'],50000.0,обязанность оказывать квалифицировать медицинс...
16313,Фармацевт,От 1 года до 3 лет,Полный день,Полная занятость,Условия: официальное оформление по ТК РФ; гра...,"['Пользователь ПК', 'Работа в команде', 'Грамо...",50000.0,условие официальный оформление тк рф график ра...
16314,Геодезист (автодороги),От 1 года до 3 лет,Вахтовый метод,Полная занятость,Обязанности: своевременное и качественное вып...,[],75000.0,обязанность своевременный качественный выполне...


In [61]:
X_train, X_valid, y_train, y_valid = train_test_split(
    train_df.drop(['salary', 'key_skills', "description"], axis=1),
    additional_train.salary_log,
    train_size=0.8,
    random_state=seed
)

In [64]:
train_pool = Pool(
    data=X_train,
    label=y_train,
    text_features=['cleaned', "name"],
    cat_features=cat_features
)

valid_pool = Pool(
    data=X_valid, 
    label=y_valid,
    text_features=['cleaned', 'name'],
    cat_features=cat_features
)

*Параметры были получены с помощью Optuna*

In [ ]:
cat1 = CatBoostRegressor(
    iterations=5000,
    eval_metric='R2',
    **{'learning_rate': 0.06087680848049949,
        'bootstrap_type': 'Bayesian',
        'boosting_type': 'Plain',
        'l2_leaf_reg': 3.5,
        'bagging_temperature': 0.5944142566635626},

    tokenizers=[
        {
            'tokenizer_id': 'Sense',
            'separator_type': 'BySense',
            'lowercasing': 'True',
            'token_types':['Word', 'Number', 'SentenceBreak'],
            'sub_tokens_policy':'SeveralTokens'
        }      
    ],
    dictionaries = [
        {
            'dictionary_id': 'Word',
            'max_dictionary_size': '50000'
        }
    ],
    feature_calcers = [
        'BoW:top_tokens_count=5000'
    ],
    random_seed=seed,
    task_type="GPU",
    devices='0:1'
)

In [ ]:
cat1.fit(train_pool,
        eval_set=valid_pool,
        verbose=100,
        use_best_model=True
)

In [ ]:
cat2 = CatBoostRegressor(
    iterations=5000,
    eval_metric='R2',
    **{'learning_rate': 0.06087680848049949,
        'bootstrap_type': 'Bayesian',
        'boosting_type': 'Plain',
        'l2_leaf_reg': 3.5,
        'bagging_temperature': 0.5944142566635626},

    tokenizers=[
        {
            'tokenizer_id': 'Sense',
            'separator_type': 'BySense',
            'lowercasing': 'True',
            'token_types':['Word', 'Number', 'SentenceBreak'],
            'sub_tokens_policy':'SeveralTokens'
        }      
    ],
    dictionaries = [
        {
            'dictionary_id': 'Word',
            'max_dictionary_size': '50000'
        }
    ],
    feature_calcers = [
        'BoW:top_tokens_count=10000'
    ],
    random_seed=seed,
    task_type="GPU",
    devices='0:1'
)

In [ ]:
cat2.fit(train_pool,
        eval_set=valid_pool,
        verbose=100,
        use_best_model=True
)

In [ ]:
def get_catboost_prediction(model, test_df):
    test_pool = Pool(
        data=test_df.drop(['id', 'description', 'key_skills'], axis=1),
        label=None,
        text_features=['cleaned', 'name'],
        cat_features=cat_features
    )
    result = model.predict(test_pool)
    result_df = pd.concat([test_df.id, pd.Series(result)], axis=1)
    result_df.columns = ['id', 'salary']
    result_df.salary = np.exp(result_df.salary)
    return result_df

In [ ]:
result_1 = get_catboost_prediction(cat1, test_df)
result_1.to_csv("/kaggle/working/prediction_1.csv", index=False)

In [ ]:
result_2 = get_catboost_prediction(cat2, test_df)
result_2.to_csv("/kaggle/working/prediction_2.csv", index=False)

## Stacking over two catboosts

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr_pool = Pool(
    train_df.drop(['salary', 'key_skills', "description"], axis=1),
    additional_train.salary_log,
    text_features=['cleaned', "name"],
    cat_features=cat_features
)

In [ ]:
lr = LinearRegression()
X_train = pd.concat([pd.Series(cat1.predict(lr_pool)), pd.Series(cat1.predict(lr_pool))], axis=1)
y_train = additional_train.salary_log
lr.fit(X_train, y_train)

In [ ]:
r2_score(y_train, lr.predict(X_train))

In [ ]:
result_combined = lr.predict(pd.concat([result_1.salary, result_2.salary], axis=1))
result_combined = pd.DataFrame(result_combined)
result_combined.columns=['salary']
pd.concat([test_df.id, result_combined], axis=1).to_csv("/kaggle/working/prediction_combined.csv", index=False)

In [65]:
# На private -> 0.51715